In [13]:
import pandas as pd
import pymysql
from sqlalchemy import create_engine

In [14]:
df = pd.read_csv('universities-clean.csv')

In [15]:
df

,Country,University,Founded,Type,Enrollment,Link
0,Albania,University of Tirana,1957,Public,35000,https://en.wikipedia.org/wiki/University_of_Ti...
1,Algeria,Constantine University,1978,Public,85000,https://en.wikipedia.org/wiki/List_of_universi...
2,Angola,Agostinho Neto University,1962,Public,29827,https://en.wikipedia.org/wiki/Agostinho_Neto_U...
3,Argentina,University of Buenos Aires,1821,Public,311175,https://en.wikipedia.org/wiki/University_of_Bu...
4,Australia,Monash University,1958,Public,73807,https://en.wikipedia.org/wiki/Monash_University
...,...,...,...,...,...,...
66,Turkey,Anadolu University,1958,Public,1969733,https://en.wikipedia.org/wiki/Anadolu_University
67,United Kingdom,Open University,1969,Public,253075,https://en.wikipedia.org/wiki/Open_University
68,United States,Arizona State University,1885,Public,194000,https://en.wikipedia.org/wiki/Arizona_State_Un...
69,Uruguay,University of the Republic,1949,Public,144108,https://en.wikipedia.org/wiki/University_of_th...


In [16]:
conn = pymysql.Connect(
    host='localhost',
    port=3307,
    password='Wikipediauserpassword',
    user='Wikipediauser',
    database='Wikipediadb'
)

In [17]:
cursor = conn.cursor()

In [18]:
cursor.execute("SHOW TABLES;")

1

In [19]:
cursor.fetchall()

(('university',),)

In [20]:
create_query_file = open('./sql/mysql_create_table.sql')
create_query = create_query_file.read()
create_query

'CREATE TABLE university (\n    id BIGINT AUTO_INCREMENT NOT NULL,\n    country VARCHAR(255) NOT NULL,\n    `name` VARCHAR(255) NOT NULL,\n    founded INT NOT NULL,\n    `type` VARCHAR(255) NOT NULL,\n    enrollment BIGINT NOT NULL,\n    link VARCHAR(255) NOT NULL,\n    UNIQUE KEY unique_combination (`country`, `name`),\n    PRIMARY KEY (id)\n);'

In [23]:
cursor.execute(create_query)

0

In [22]:
df = pd.read_csv('universities-clean.csv')
data = list(df.itertuples(index=None, name=None))

In [26]:
data[:2]

[('Albania',
  'University of Tirana',
  1957,
  'Public',
  35000,
  'https://en.wikipedia.org/wiki/University_of_Tirana'),
 ('Algeria',
  'Constantine University',
  1978,
  'Public',
  85000,
  'https://en.wikipedia.org/wiki/List_of_universities_in_Algeria')]

In [24]:
engine = create_engine("mysql+pymysql://", creator=lambda: conn)

In [25]:
pd.read_sql("SELECT * FROM university;", con=engine)

,id,country,name,founded,type,enrollment,link


In [27]:
merge_query_file = open('./sql/mysql_upsert.sql')
merge_query = merge_query_file.read()
merge_query

'INSERT INTO university (country, `name`, founded, `type`, enrollment, link)\nVALUES (%s, %s, %s, %s, %s, %s)\nON DUPLICATE KEY UPDATE\n    founded=VALUES(founded),\n    `type`=VALUES(`type`),\n    enrollment=VALUES(enrollment),\n    link=VALUES(link);\n'

In [28]:
cursor.executemany(merge_query, data)

71

In [29]:
conn.commit()

In [30]:
pd.read_sql("SELECT * FROM university;", con=engine)

,id,country,name,founded,type,enrollment,link
0,1,Albania,University of Tirana,1957,Public,35000,https://en.wikipedia.org/wiki/University_of_Ti...
1,2,Algeria,Constantine University,1978,Public,85000,https://en.wikipedia.org/wiki/List_of_universi...
2,3,Angola,Agostinho Neto University,1962,Public,29827,https://en.wikipedia.org/wiki/Agostinho_Neto_U...
3,4,Argentina,University of Buenos Aires,1821,Public,311175,https://en.wikipedia.org/wiki/University_of_Bu...
4,5,Australia,Monash University,1958,Public,73807,https://en.wikipedia.org/wiki/Monash_University
...,...,...,...,...,...,...,...
66,67,Turkey,Anadolu University,1958,Public,1969733,https://en.wikipedia.org/wiki/Anadolu_University
67,68,United Kingdom,Open University,1969,Public,253075,https://en.wikipedia.org/wiki/Open_University
68,69,United States,Arizona State University,1885,Public,194000,https://en.wikipedia.org/wiki/Arizona_State_Un...
69,70,Uruguay,University of the Republic,1949,Public,144108,https://en.wikipedia.org/wiki/University_of_th...
